In [1]:
import sys
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import string
import numpy as np
import re
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from collections import OrderedDict
import torch.optim as optim
import time

In [2]:
book = open('AlicesAdvanturesInWonderland.txt', 'r')
book = book.read()
book = book[715:145060]
book_edit = re.sub('[+]', '', book)
book_edit = re.sub(r'(CHAPTER \w+.\s)', '', book)
book_edit = book_edit.lower()

words = word_tokenize(book_edit)
words = [word.split("'") for word in words]

for w in words:
    for v in w:
        if v in string.punctuation:
            w.remove(v)
            
filtered_words = []

for word in words:
    for v in word:
        if v != "" and v != "--":
            filtered_words.append(v)
    
filtered_words[:15]

['down',
 'the',
 'rabbit-hole',
 'alice',
 'was',
 'beginning',
 'to',
 'get',
 'very',
 'tired',
 'of',
 'sitting',
 'by',
 'her',
 'sister']

In [3]:
#vocab to int / int to vocab mapping
word_2_idx = {}
idx_2_word = {}

for word in filtered_words:
    if word not in word_2_idx:
        word_2_idx[word] = len(word_2_idx)
        idx_2_word[len(idx_2_word)] = word

print("Total number of words: {} / Number of unique words: {}".format(len(filtered_words), len(word_2_idx)))

Total number of words: 27414 / Number of unique words: 2595


In [4]:
#Vectorize the dataset based off word_2_idx mapping
vectorized_words = []

for i in filtered_words:
    if i in word_2_idx.keys():
        vectorized_words.append(word_2_idx[i])

In [5]:
sequence_length = 4

def make_features(text, seq_len):
    return [text[i:(i + seq_len)] for i in range(math.ceil(len(text)/seq_len))]

def make_targets(text, seq_len):
    return [text[(i + seq_len)] for i in range(math.ceil(len(text)/seq_len))]

feature_tensors = make_features(vectorized_words, sequence_length)
feature_tensors = torch.FloatTensor(feature_tensors)
target_tensors = make_targets(vectorized_words, sequence_length)
target_tensors = torch.FloatTensor(target_tensors)

print(type(feature_tensors))

<class 'torch.Tensor'>


In [6]:
#BATCHING DATA - SPLIT DATA INTO BATCHES OF ?? AS FEATURE TENSORS
#TARGET TENSOR WOULD BE THE NEXT WORD AFTER THE BATCH END
from torch.utils.data import TensorDataset, DataLoader

batch_size = 10

data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)


In [7]:
#CHECKING THE DATALOADER
data_iter = iter(data_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 4])
tensor([[ 0.,  1.,  2.,  3.],
        [ 1.,  2.,  3.,  4.],
        [ 2.,  3.,  4.,  5.],
        [ 3.,  4.,  5.,  6.],
        [ 4.,  5.,  6.,  7.],
        [ 5.,  6.,  7.,  8.],
        [ 6.,  7.,  8.,  9.],
        [ 7.,  8.,  9., 10.],
        [ 8.,  9., 10., 11.],
        [ 9., 10., 11., 12.]])

torch.Size([10])
tensor([ 4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.])


In [8]:
#define model
n_input = sample_x.shape[0] * sample_x.shape[1]
hidden_layers = [256, 512]
n_output = 1

class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        super().__init__()
        
        self.hidden1 = nn.Linear(input_size, hidden_layers[0])
        self.hidden2 = nn.Linear(hidden_layers[0], hidden_layers[1])
        self.output = nn.Linear(hidden_layers[1], output_size)
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.dropout(x)
        x = F.relu(self.hiddenx(x))
        x = self.dropout(x)
        x = self.output(x)
        return F.log_softmax(x, dim=1)
        
model = Network(n_input, n_output, hidden_layers)

model

Network(
  (hidden1): Linear(in_features=40, out_features=256, bias=True)
  (hidden2): Linear(in_features=256, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=1, bias=True)
)

In [9]:
#NEED TO FLATTEN THE FEATURES BEFORE MOVING FORWARD
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 2
steps = 0
running_loss= 0
print_every= 40

for e in range(epochs):
    model.train()
    for feature, label in data_loader:
        steps += 1
        # RESIZE THE FEATURES TO A FLAT TENSOR --- feature.resize_(feature.size([0]), -1)
        optimizer.zero_grad()
        output = model.forward(feature)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if steps % print_every == 0:
            model.eval()
            print(running_loss)            

RuntimeError: size mismatch, m1: [10 x 4], m2: [40 x 256] at C:\w\1\s\tmp_conda_3.7_104508\conda\conda-bld\pytorch_1572950778684\work\aten\src\TH/generic/THTensorMath.cpp:197